In [ ]:
!pip install torch torchvision transformers datasets accelerate scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [ ]:
import os
import json
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from collections import Counter

from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms

from transformers import (
    ViTImageProcessor,
    ViTForImageClassification,
    Trainer,
    TrainingArguments,
    EvalPrediction
)

In [ ]:
from google.colab import drive
from tqdm import tqdm

# ✅ Mount Drive
print("\n📂 Mounting Google Drive...")
drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/MultiBanFake/Dataset'
TRAIN_DIR = os.path.join(BASE_PATH, 'Train')
VAL_DIR   = os.path.join(BASE_PATH, 'Validation')
TEST_DIR  = os.path.join(BASE_PATH, 'Test')


📂 Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import shutil

def organize_dataset_images(split_dir):
    print(f"🔁 Organizing: {split_dir}")

    for fname in os.listdir(split_dir):
        fpath = os.path.join(split_dir, fname)

        if not os.path.isfile(fpath):
            continue  # skip if not a file

        fname_lower = fname.lower()
        if "fake" in fname_lower:
            label = "Fake"
        elif "real" in fname_lower:
            label = "Real"
        else:
            print(f"⚠️ Skipped (no label found): {fname}")
            continue

        label_dir = os.path.join(split_dir, label)
        os.makedirs(label_dir, exist_ok=True)

        shutil.move(fpath, os.path.join(label_dir, fname))
        print(f"✅ Moved: {fname} → {label}/")

# 🔁 Paths to your dataset folders in Drive
base_path = "/content/drive/MyDrive/MultiBanFake/Dataset"
splits = ["Train", "Validation", "Test"]

for split in splits:
    organize_dataset_images(os.path.join(base_path, split))


Streaming output truncated to the last 5000 lines.
✅ Moved: international_fake_155.png → Fake/
✅ Moved: international_fake_20.png → Fake/
✅ Moved: international_fake_186.png → Fake/
✅ Moved: international_fake_161.png → Fake/
✅ Moved: international_fake_19.png → Fake/
✅ Moved: international_fake_185.png → Fake/
✅ Moved: international_fake_180.png → Fake/
✅ Moved: international_fake_178.png → Fake/
✅ Moved: international_fake_201.jpg → Fake/
✅ Moved: international_fake_151.png → Fake/
✅ Moved: international_fake_164.png → Fake/
✅ Moved: international_fake_141.png → Fake/
✅ Moved: international_fake_176.png → Fake/
✅ Moved: international_fake_173.png → Fake/
✅ Moved: international_fake_159.png → Fake/
✅ Moved: international_fake_163.png → Fake/
✅ Moved: international_fake_16.png → Fake/
✅ Moved: international_fake_150.png → Fake/
✅ Moved: international_fake_187.png → Fake/
✅ Moved: international_fake_191.png → Fake/
✅ Moved: international_fake_171.png → Fake/
✅ Moved: international_fake_

In [ ]:
import os
from collections import Counter

def count_images_per_class(split_path):
    class_counts = {}
    for label in os.listdir(split_path):
        label_path = os.path.join(split_path, label)
        if os.path.isdir(label_path):
            num_images = len([
                f for f in os.listdir(label_path)
                if os.path.isfile(os.path.join(label_path, f))
                and f.lower().endswith(('.png', '.jpg', '.jpeg', '.webp'))
            ])
            class_counts[label] = num_images
    return class_counts

splits = ["Train", "Validation", "Test"]
base_path = "/content/drive/MyDrive/MultiBanFake/Dataset"

for split in splits:
    split_path = os.path.join(base_path, split)
    class_distribution = count_images_per_class(split_path)
    print(f"📊 Class distribution in {split}: {class_distribution}")


📊 Class distribution in Train: {'Fake': 3840, 'Real': 3840}
📊 Class distribution in Validation: {'Fake': 477, 'Real': 480}
📊 Class distribution in Test: {'Fake': 477, 'Real': 480}


In [ ]:
from transformers import AutoImageProcessor

# ✅ Define transforms using Swin processor
processor = AutoImageProcessor.from_pretrained("microsoft/swin-tiny-patch4-window7-224")
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=processor.image_mean, std=processor.image_std),
])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# ✅ Load datasets
train_dataset = ImageFolder(root=TRAIN_DIR, transform=transform)
val_dataset   = ImageFolder(root=VAL_DIR, transform=transform)
test_dataset  = ImageFolder(root=TEST_DIR, transform=transform)

label_names = train_dataset.classes
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {label: i for i, label in enumerate(label_names)}

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return {"pixel_values": image, "labels": label}

train_ds = CustomDataset(train_dataset)
val_ds   = CustomDataset(val_dataset)
test_ds  = CustomDataset(test_dataset)

In [ ]:
# ✅ Define compute_metrics
from sklearn.metrics import classification_report

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    report = classification_report(labels, preds, target_names=label_names, output_dict=True)
    print(json.dumps(report, indent=2))
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": report['weighted avg']['f1-score'],
    }

In [ ]:
from transformers import AutoModelForImageClassification

model = AutoModelForImageClassification.from_pretrained(
    "microsoft/swin-tiny-patch4-window7-224",
    num_labels=len(label_names),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True  # ✅ Fixes classification head mismatch
)

model.gradient_checkpointing_enable()


Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-tiny-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./swin_results",                     # saves checkpoints here
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,
    do_train=True,
    do_eval=True,
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=processor,
    compute_metrics=compute_metrics
)

<ipython-input-8-443653571>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
#15a26bfde02e39ba547185222150f57c5c5d8e02
from transformers import TrainerCallback
from transformers.trainer_utils import get_last_checkpoint

checkpoint = get_last_checkpoint("./swin_results")

if checkpoint:
    print(f"🔄 Resuming from checkpoint: {checkpoint}")
    trainer.train(resume_from_checkpoint=checkpoint)
else:
    print("🚀 Starting training from scratch...")
    trainer.train()


🚀 Starting training from scratch...


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zarin-cse-20210104111 (zarin-cse-20210104111-ahsanullah-university-of-science-t) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.729800
20,0.705300
30,0.668200
40,0.712200
50,0.704900
60,0.714000
70,0.676100
80,0.695000
90,0.684900
100,0.688300


In [ ]:
# ✅ Save model
model.save_pretrained("/content/swin_final_model")

In [ ]:
from sklearn.metrics import classification_report
from collections import Counter
import torch
import json

# Step 1: Run prediction
preds = trainer.predict(test_ds)
pred_labels = np.argmax(preds.predictions, axis=1)
true_labels = preds.label_ids

# Step 2: Print predicted class counts
pred_counts = dict(Counter(pred_labels))
print("🔍 Predicted Class Distribution:", pred_counts)

# Step 3: Classification report
report = classification_report(
    true_labels,
    pred_labels,
    target_names=label_names,
    digits=4,
)
print("📊 Fake vs Non-Fake Class Report:\n", report)

# Step 4: Save report and predictions
with open("swin_classification_report.json", "w") as f:
    json.dump(report, f, indent=4)

# Save softmax probabilities for ensembling
probs = torch.nn.functional.softmax(torch.tensor(preds.predictions), dim=-1).numpy().tolist()

with open("swin_stack_probs.json", "w") as f:
    json.dump({
        "swin_probs": probs,
        "true_labels": true_labels.tolist()
    }, f)

print("✅ Saved classification report and stacking probabilities.")


{
  "Fake": {
    "precision": 0.6200873362445415,
    "recall": 0.5953878406708596,
    "f1-score": 0.6074866310160428,
    "support": 477.0
  },
  "Real": {
    "precision": 0.6132264529058116,
    "recall": 0.6375,
    "f1-score": 0.6251276813074566,
    "support": 480.0
  },
  "accuracy": 0.6165099268547545,
  "macro avg": {
    "precision": 0.6166568945751765,
    "recall": 0.6164439203354297,
    "f1-score": 0.6163071561617497,
    "support": 957.0
  },
  "weighted avg": {
    "precision": 0.616646140839536,
    "recall": 0.6165099268547545,
    "f1-score": 0.6163348067107959,
    "support": 957.0
  }
}
🔍 Predicted Class Distribution: {np.int64(1): 499, np.int64(0): 458}
📊 Fake vs Non-Fake Class Report:
               precision    recall  f1-score   support

        Fake     0.6201    0.5954    0.6075       477
        Real     0.6132    0.6375    0.6251       480

    accuracy                         0.6165       957
   macro avg     0.6167    0.6164    0.6163       957
weighted